
There are two methods:
1.You can use the magic command %notebook to save as ipynb

  %notebook "directory/to/file/filename.ipynb"     
2.You can use the magic command %%writefile to save as py file

  %%writefile "directory/to/file/filename.py"
In the second method, you should put this command at the top of the cell, otherwise it will through an error.



In [69]:
import pandas as pd
import yfinance as yf
import datetime 

aapl = yf.Ticker("AAPL")
flat_df = pd.DataFrame()
for expi in aapl.options:
    chain = aapl.option_chain(expi)
    chain.calls["putcall"] = "call"
    chain.puts["putcall"] = "put"

    flat_df = flat_df.append(chain.calls)
    flat_df = flat_df.append(chain.puts)
    flat_df["expiry"] = datetime.datetime.strptime(expi, '%Y-%m-%d')

flat_df["undticker"] = "AAPL"
flat_df = flat_df[["contractSymbol", "strike", "undticker", "expiry", "putcall"]]
flat_df = flat_df.rename(columns={'contractSymbol': 'ticker'})

print flat_df.head(10)

                ticker  strike undticker     expiry putcall
0  AAPL200103C00225000   225.0      AAPL 2019-12-13    call
1  AAPL200103C00235000   235.0      AAPL 2019-12-13    call
2  AAPL200103C00240000   240.0      AAPL 2019-12-13    call
3  AAPL200103C00247500   247.5      AAPL 2019-12-13    call
4  AAPL200103C00250000   250.0      AAPL 2019-12-13    call
5  AAPL200103C00252500   252.5      AAPL 2019-12-13    call
6  AAPL200103C00255000   255.0      AAPL 2019-12-13    call
7  AAPL200103C00257500   257.5      AAPL 2019-12-13    call
8  AAPL200103C00260000   260.0      AAPL 2019-12-13    call
9  AAPL200103C00262500   262.5      AAPL 2019-12-13    call


In [70]:
flat_df.to_json(os.path.join(os.getcwd(),"yahoo.json"), orient='records', date_format='epoch')

In [71]:
import simplejson as json
import requests

contracts = json.loads(flat_df.to_json(orient='records', date_format='epoch'))
res = [requests.post('http://localhost:3000/europeans', json=contract) for contract in contracts]

print res.head(10)

AttributeError: 'list' object has no attribute 'head'

In [ ]:
res = requests.get('http://localhost:3000/europeans')
europeans = res.json()['europeans']

print json.dumps(europeans[:20], indent=4)